# Aprendizaje supervisado - Árboles de clasificación
![Árbol de clasificación](images/ilustracion_arb_clasi.png)

## Carga del conjunto de datos

In [ ]:
# Importar modulos para el tratamiento de los datos y visualizaciones
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Cargar los datos en un dataframe de Pandas
url = 'https://raw.githubusercontent.com/JASDataCTG/Diplomado-ML/main/Modulo%203/Datasets/Movie_classificationesclas.csv'
df = pd.read_csv(url, header=0)

In [ ]:
df.head()

In [ ]:
df.info()

### Imputación de valores faltantes

In [ ]:
df['Tiempo realizacion'].mean()

In [ ]:
# Imputar los valores faltantes con la media de los valores existentes
df['Tiempo realizacion'].fillna(value = df['Tiempo realizacion'].mean(), inplace = True)

In [ ]:
df.head()

In [ ]:
df.info()

### Creación de variables dummy

In [ ]:
list(dict.fromkeys(df['Genero']))

In [ ]:
# Crear variables dummy eliminando una de ellas y el atributo que la origina
df = pd.get_dummies(df, columns = ['Disponibilidad 3D', 'Genero'], drop_first = True)

In [ ]:
df.head()

### División en la variable predicha y el vector de características

In [ ]:
X = df.loc[:, df.columns!= 'Gana Oscar']

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y = df['Gana Oscar']

In [ ]:
y.head()

In [ ]:
y.shape

### Creación del conjunto de entrenamiento y validación

In [ ]:
# Cargar el modulo para dividir el conjunto de datos en conjunto de entrenamiento y validación
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Entrenamiento del árbol de clasificación

In [ ]:
# Importar modulo para la creación de los árboles
from sklearn import tree

In [ ]:
arbolclas = tree.DecisionTreeClassifier(max_depth = 3)

In [ ]:
arbolclas.fit(X_train, y_train)

### Predicción en el conjunto de entrenamiento y validación

In [ ]:
y_train_pred = arbolclas.predict(X_train)
y_test_pred = arbolclas.predict(X_test)

In [ ]:
y_test_pred

### Métricas del modelo

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
confusion_matrix(y_test, y_test_pred)

In [ ]:
plot_confusion_matrix(conf_mat = confusion_matrix(y_train, y_train_pred), figsize=(6,6), show_normed=False)
plt.tight_layout(pad = 10)

In [ ]:
plot_confusion_matrix(conf_mat = confusion_matrix(y_test, y_test_pred), figsize=(6,6), show_normed=False)
plt.tight_layout(pad = 10)

In [ ]:
accuracy_score(y_test, y_test_pred)

In [ ]:
roc_auc_score(y_test, y_test_pred)

### Gráficar árbol de regresión

In [ ]:
# Utilizar el software graphviz para dar formato al árbol
datos_graf = tree.export_graphviz(arbolclas, out_file = None, feature_names = X_train.columns,
                                  filled = True, class_names = True)

In [ ]:
# Del shell interactivo Ipython importar las funciones para crear la imagen
from IPython.display import Image

In [ ]:
# Importar modulo que genera la estructura del árbol y graficarlo de acuerdo a los datos
import pydotplus

In [ ]:
# Crear la imagen en formato png
graph = pydotplus.graph_from_dot_data(datos_graf)
Image(graph.create_png())
#graph.write_png('arboloriginalclass.png') # Si se desea guardar el árbol

### Controlando el crecimiento del árbol

In [ ]:
arbolclas1 = tree.DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 20, min_samples_split = 15)
arbolclas1.fit(X_train, y_train)
datos_graf = tree.export_graphviz(arbolclas1, out_file = None, feature_names = X_train.columns,
                                  filled = True, class_names = ['No Gana', 'Gana'])
graph1 = pydotplus.graph_from_dot_data(datos_graf)
Image(graph1.create_png())

# Métodos de ensemble
Los métodos de Bagging, Random Forests y Boosting permiten obtener una mejora sustancial en el rendimiento predictivo de los modelos basados en árboles, son modelos del tipo caja negra que son difíciles de interpretar o imposibles de interpretar en cierto nivel. Se conocen estos como métodos de ensemble o métodos combinados. Son aplicables a otros métodos de aprendizaje estadísticos para labores de regresión o clasificación.

Los algoritmos ensamblados son algoritmos formados por algoritmos más simples. Estos algoritmos simples se unen para formar un algoritmo más potente.

### Baggin
El método de bagging o bootstrap aggregation es un procedimiento utilizado para reducir la varianza de un método de aprendizaje estadístico, usado muy frecuentemente con árboles de decisión.

Para árboles de regresión la aplicación del bagging consiste en crear B árboles de regresión usando los B sets de entrenamiento generados por bootstrapping, promediando finalmente las predicciones resultantes. Estos árboles pueden crecer bastante ya que apenas se aplican restricciones, además de que no son podados. De esta manera cada árbol individual tiene alta varianza y poco bias, pero promediando los B árboles se contrarresta la varianza.

Para árboles de clasificación dada una observación de validación, podemos obtener la clase predicha por cada uno de los B árboles, y escoger como predicción final para dicha observación la clase más común de entre las B predicciones (predicción de cada árbol).
![Formula Bagging](images/bagging.png)

![Concepto de bootstrap](images/bootrap_concept.png)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

In [ ]:
# Importar modulo para Bagging
from sklearn.ensemble import BaggingClassifier

In [ ]:
clas_bag = BaggingClassifier(base_estimator = arbolclas, n_estimators = 1000, bootstrap = True, n_jobs = -1, random_state = 123)

In [ ]:
clas_bag.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, clas_bag.predict(X_test))

In [ ]:
accuracy_score(y_test, clas_bag.predict(X_test))

### Random Forest
Todos los árboles generados por Bagging usarán un predictor muy influyente en la primera ramificación, por lo que serán similares unos a otros, y las predicciones entre ellos estarán altamente correlacionadas, por lo tanto, la aplicación de Bagging promediando valores correlacionados no consigue una reducción sustancial de la varianza con respecto a un solo árbol.

El método de Random Forests proporciona una mejora a los árboles combinados por Bagging en cuanto a que los descorrelaciona, teniendo en cuenta solo un subgrupo de predictores en cada división. Se construyen un número de árboles de decisión a partir de pseudo-muestras generadas por bootstrapping. Esta vez, se escogen de entre todos los $p$ predictores una muestra aleatoria de m predictores como candidatos antes de cada división, generalmente $m=\sqrt{p}$ (si $m=p$, bagging y random forests darían resultados equivalentes).

![RF1](images/rf1.png)
![RF2](images/rf2.png)
![RF3](images/rf3.png)
![RF4](images/rf4.png)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforestclassifier#sklearn.ensemble.RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clas_rf = RandomForestClassifier(n_estimators = 1000, n_jobs = -1, random_state = 123)

In [ ]:
clas_rf.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, clas_rf.predict(X_test))

In [ ]:
accuracy_score(y_test, clas_rf.predict(X_test))

#### Grid Search
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV

In [ ]:
# Importar GridSearch para la prueba y elección de mejores estimaciones
from sklearn.model_selection import GridSearchCV

In [ ]:
parametros = {'max_features' : [4, 5, 6, 7, 8, 9, 10], 'min_samples_split' : [2, 3, 10]}

In [ ]:
pru_param = GridSearchCV(clas_rf, parametros, n_jobs = -1, cv = 10, scoring = 'accuracy')

In [ ]:
pru_param.fit(X_train, y_train)

In [ ]:
pru_param.best_params_

In [ ]:
best_rf = pru_param.best_estimator_

In [ ]:
confusion_matrix(y_test, best_rf.predict(X_test))

In [ ]:
accuracy_score(y_test, best_rf.predict(X_test))

### Boosting
Boosting funciona de manera parecida al bagging en cuanto a que combina un gran número de árboles, a excepción de que los árboles se construyen de manera secuencial: cada árbol se genera usando información, concretamente los residuos, de árboles previamente generados, en lugar de utilizar la variable respuesta (por ello suelen ser suficientes árboles más pequeños, en lugar de un gran árbol que pueda sobreajustarse a los datos). Otra diferencia es que boosting no utiliza remuestreo por bootstrapping, sino que cada árbol se genera utilizando una versión modificada del set de datos original.

Tres de los algoritmos de boosting más empleados son AdaBoost, Gradient Boosting y Stochastic Gradient Boosting. Todos ellos se caracterizan por tener una cantidad considerable de hiperparámetros, cuyo valor óptimo se tiene que identificar mediante validación cruzada. Tres de los más importantes son:

* El número de weak learners o número de iteraciones: a diferencia del bagging y random forest, el boosting puede sufrir overfitting si este valor es excesivamente alto. Para evitarlo, se emplea un término de regularización conocido como learning rate.

* Learning rate: controla la influencia que tiene cada weak learner en el conjunto del ensemble, es decir, el ritmo al que aprende el modelo. Suelen recomendarse valores de 0.001 o 0.01, aunque la elección correcta puede variar dependiendo del problema. Cuanto menor sea su valor, más árboles se necesitan para alcanzar buenos resultados pero menor es el riesgo de overfitting.

* Si los weak learners son árboles, el tamaño máximo permitido de cada árbol. Suelen emplearse valores pequeños, entre 1 y 10.

Estos métodos de Boosting tienen a disminuir el error con la desventaja de que el modelo resultante se sobreajuste a los datos de entrenamiento, esto lo logran disminuyendo el sesgo y la varianza:

$Error total=sesgo+varianza+\epsilon$

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html?highlight=gradientboostingclassifier#sklearn.ensemble.GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clas_bo = GradientBoostingClassifier()
clas_bo.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clas_bo.predict(X_test))

In [ ]:
clas_bo2 = GradientBoostingClassifier(learning_rate = 0.02, n_estimators = 1000, max_depth = 1)
clas_bo2.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clas_bo.predict(X_test))

#### Ada Boost
Adaboost (Adaptive Boosting), aplicado sobretodo a problemas de clasificación, es un algoritmo iterativo que se basa en combinar múltiples weak learners, en un único strong learner a través de una combinación lineal ponderada.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html?highlight=adaboostclassifier#sklearn.ensemble.AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clas_ab = AdaBoostClassifier(learning_rate = 0.02, n_estimators = 5000)

In [ ]:
clas_ab.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clas_ab.predict(X_test))

In [ ]:
clas_ab2 = AdaBoostClassifier(learning_rate = 0.02, n_estimators = 500)

In [ ]:
clas_ab2.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clas_ab2.predict(X_test))

#### XG Boost
Gradient Boosting no asigna un peso independiente a cada observación de entrenamiento, sino que hace uso de una función de coste $L(yi,f(x))$ cuyo gradiente o derivada parcial de la función de coste se pretende minimizar. Esto se lleva a cabo en un proceso iterativo. El gradiente se utiliza para encontrar la dirección en la que cambiar los parámetros de los weak learners para reducir el error de predicción en las siguientes iteraciones. Concretamente, las predicciones del weak learner m intentan acercarse al gradiente negativo de la función de coste.

![Gradiente](images/grad1.png)

https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
import xgboost as xgb

In [ ]:
clas_xgb = xgb.XGBClassifier(max_depth = 5, n_estimators = 10000, learning_rate = 0.3, n_jobs = -1)

In [ ]:
clas_xgb.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, clas_xgb.predict(X_test))

In [ ]:
xgb.plot_importance(clas_xgb)

In [ ]:
clas_xgb2 = xgb.XGBClassifier(n_estimators = 500, learning_rate = 0.1, n_jobs = -1, random_state = 123)

In [ ]:
parametros2 = {'max_depth': range(3, 10, 2),
               'gamma' : [0.1, 0.2, 0.3],
               'sub_sample' : [0.8, 0.9],
              'colsample_bytree' : [0.8, 0.9],
              'reg_alpha' : [1e-2, 0.1, 1]}

In [ ]:
pru_param2 = GridSearchCV(clas_xgb2, parametros2, n_jobs = -1, cv = 10, scoring = 'accuracy')

In [ ]:
pru_param2.fit(X_train, y_train)

In [ ]:
best_xgb = pru_param2.best_estimator_

In [ ]:
accuracy_score(y_test, best_xgb.predict(X_test))

In [ ]:
pru_param2.best_params_